In [1]:
import gym
import numpy as np

In [2]:
env = gym.make('FrozenLake-v0')

In [3]:
env.render()


SFFF
FHFH
FFFH
HFFG


In [38]:
print(env.observation_space)

Discrete(16)


In [39]:
print(env.action_space)

Discrete(4)


In [10]:
env.P.keys()

dict_keys([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15])

In [12]:
env.P[1].keys()

dict_keys([0, 1, 2, 3])

In [23]:
env.P[1][1]

[(0.3333333333333333, 0, 0.0, False),
 (0.3333333333333333, 5, 0.0, True),
 (0.3333333333333333, 2, 0.0, False)]

## Value Iteration

In [65]:
def value_iteration(env, gamma=1.0, max_iterations=10000, threshold=1e-10):
    value_table = np.zeros(env.observation_space.n)
    policy = np.zeros(env.observation_space.n)
    
    for i in range(max_iterations):
        updated_value_table = np.copy(value_table)

        for state in range(env.observation_space.n):
            Q_values = np.zeros(env.action_space.n)

            for action in range(env.action_space.n):    
                next_rewards = []
                
                for trans_prob, next_state, reward, _ in env.P[state][action]: # env.P[s][a] -> list
                    next_state_reward = trans_prob * (reward + gamma * updated_value_table[next_state])
                    next_rewards.append(next_state_reward)
                    
                Q_values[action] = sum(next_rewards)

            value_table[state] = np.max(Q_values)
            policy[state] = np.argmax(Q_values)
            
        if np.sum(np.fabs(updated_value_table - value_table)) <= threshold:
            print("Value iteration converged at iteration %d." %(i+1))
            break
            
    return policy, value_table

In [66]:
optimal_policy, optimal_values = value_iteration(env=env, gamma=1.0)

Value iteration converged at iteration 877.


In [67]:
def print_graph(values, width):
    for i in range(0, len(values), width):
        print(values[i: i+width])

In [68]:
print_graph(optimal_policy, 4)

[ 0.  3.  3.  3.]
[ 0.  0.  0.  0.]
[ 3.  1.  0.  0.]
[ 0.  2.  1.  0.]


In [69]:
print_graph(optimal_values, 4)

[ 0.82352941  0.82352941  0.82352941  0.82352941]
[ 0.82352941  0.          0.52941176  0.        ]
[ 0.82352941  0.82352941  0.76470588  0.        ]
[ 0.          0.88235294  0.94117647  0.        ]


## Policy Iteration

In [60]:
def policy_evaluation(env, policy, gamma=1.0, max_iteration=10000, threshold=1e-10):
    value_table = np.zeros(env.observation_space.n)
    
    for i in range(max_iteration):
        updated_value_table = np.copy(value_table)
        
        for state in range(env.observation_space.n):
            action = policy[state]
            next_rewards = []
            
            for trans_prob, next_state, reward, _ in env.P[state][action]: 
                next_state_reward = trans_prob * (reward + gamma * updated_value_table[next_state])
                next_rewards.append(next_state_reward)
                
            Q_action = sum(next_rewards)
            value_table[state] = Q_action
        
        if np.sum(np.fabs(updated_value_table - value_table)) <= threshold:
            print("Policy evaluation converged at iteration %d." %(i+1))
            break
            
    return value_table

In [61]:
def policy_improvement(env, value_table, gamma=1.0):
    policy = np.zeros(env.observation_space.n)
    
    for state in range(env.observation_space.n):
        Q_values = np.zeros(env.action_space.n)
        
        for action in range(env.action_space.n):
            next_rewards = []
            
            for trans_prob, next_state, reward, _ in env.P[state][action]: 
                next_state_reward = trans_prob * (reward + gamma * value_table[next_state])
                next_rewards.append(next_state_reward)
                
            Q_values[action] = sum(next_rewards)
        
        policy[state] = np.argmax(Q_values)
        
    return policy

In [62]:
def policy_iteration(env, max_iteration=10000):
    policy = np.random.randint(0, 4, size=env.observation_space.n)
    
    for i in range(max_iteration):
        value_table = policy_evaluation(env, policy)
        new_policy = policy_improvement(env, value_table)
        
        if np.all(policy == new_policy):
            print("Policy iteration converged at iteration %d." %(i+1))
            break
        else:
            policy = new_policy
        
    return policy

In [63]:
optimal_policy = policy_iteration(env)

Policy evaluation converged at iteration 78.
Policy evaluation converged at iteration 534.
Policy evaluation converged at iteration 415.
Policy evaluation converged at iteration 701.
Policy evaluation converged at iteration 879.
Policy iteration converged at iteration 5.


In [64]:
print_graph(optimal_policy, 4)

[ 0.  3.  3.  3.]
[ 0.  0.  0.  0.]
[ 3.  1.  0.  0.]
[ 0.  2.  1.  0.]
